In [1]:
# Load required packages
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import scipy as sp
import os
import threading
import multiprocessing
import pickle
import keras

print "Packages loaded"


Using TensorFlow backend.


Packages loaded


In [2]:
### Functions for feature extraction
class arayi(object):
    def __init__(self):
        pass
        
    # Get list of tracks for extraction
    def getTrackList(self,pathToTrack,listBool,trackName):
        if listBool == False:
            self.filelist=[]
            with open(pathToTrack+"/metadata/perfs.csv","r") as perfData:
                perfData.next()
                for line in perfData:
                    line=line.split(",")
                    if (line[2].replace("_"," ").lstrip().rstrip() == trackName and line[6].rstrip() == "EN"):
                        self.filelist.append(line[1])
            self.filelist=tuple(self.filelist)
            return self.filelist
        else:
            self.filelist = os.listdir(pathToTrack)            
            return self.list

    # Extract Melodic Contours from Dataset
    def extractMelody(self, pathToTrack,trackList):
        self.extractStatus = 0.0
        pathToTrack = pathToTrack+"raw/"
        # Extracts contour for a track snd saves output for each track
        for track in trackList:
            self.trackOut = "/Users/nus/singability_data/contours/%s.yml" % track.split(".")[0]
            os.system("essentia_streaming_predominantpitchmelodia %s %s" % (pathToTrack+track+".m4a", self.trackOut))
            if float(trackList.index(track)+1)/len(trackList) > self.extractStatus:
                print "Extracting...%s %% complete." % (self.extractStatus*100)
                self.extractStatus += 0.33

In [3]:
## Threaded feature extraction - Needs work
def threadMelodyExtract(self,pathToTrack,trackList):
    ##TODO: Threading locks
    threads = []
    t=[]
    for track in trackList:
        if ".m4a" not in track or "_" not in track:
            continue
        t = threading.Thread(target=a.extractMelody, args=(pathToTrack,track,))
        threads.append(t)
        if len(threads) > 9:
            t.start()
            t.join()
            t=[]
            threads=[]

# def threadMelodyExtract2(self,pathToTrack,trackList):
#     ##TODO: Threading locks
#     lock = threading.Lock()
#     threads = []
#     for track in trackList:
#         if ".m4a" not in track or "_" not in track:
#             continue
#         t = threading.Thread(target=a.extractMelody, args=(pathToTrack,track,))
#         lock.acquire()
#         if not lock.acquire(False):
#             print "Fail"
#         else:
#             try:
#                 t.start()
#                 t.join()
#         finally:
#             lock.release()
            
# arayi.threadMelodyExtract = threadMelodyExtract
# arayi.threadMelodyExtract2 = threadMelodyExtract2

In [5]:
def loadContourData(self,pathToContours):
    cList=os.listdir(pathToContours)
    self.contourMatrix = []
    for contour in cList:
        if ".yml" not in contour:
            continue
        x=list(open(pathToContours+contour,"r"))
        x=map(float,x[7].lstrip().replace("pitch: ","").replace("[","").replace("]","").split(","))
        self.contourMatrix.append(x)
    df = pd.DataFrame(self.contourMatrix,columns=None)
    return df

def excludeData(self,contourList):
    # THIS FUNCTION WILL CHECK ALL OF THE COMPLETED CONTOURS TO A LIST OF APPLICABLE SONGS.
    pass

arayi.loadContourData = loadContourData
a=arayi()
df=a.loadContourData("/Users/nus/singability_data/contours/")

In [15]:
#df.to_pickle("/Users/nus/Documents/Github/Singability/letItGo.pkl")
#df = pickle.load(open("/Users/nus/Documents/Github/Singability/letItGo.pkl","rb"))
df=df.transpose()
for index, row in df.iterrows():
    row=[0.0 if x != "NaN" else x for x in row]
    print index,len(row)
    if index > 10:
        break


SyntaxError: invalid syntax (<ipython-input-15-a789f1731465>, line 5)

In [124]:
# Run Functions
pathToTrack = "/Users/nus/singability_data/"
a = arayi()
tList = a.getTrackList(pathToTrack, False, "let it go")
a.extractMelody(pathToTrack, tList)


# a.threadMelodyExtract(pathToTrack,tList)

'''
NUM_WORKERS = 4
processes = [multiprocessing.Process(target=a.extractMelody(pathToTrack,tList)) for _ in range(NUM_WORKERS)]
print "Starting Process"
[process.start() for process in processes]
print "Joining process"
[process.join() for process in processes]
'''


Extracting...0.0 % complete.
Extracting...33.0 % complete.
Extracting...66.0 % complete.
Extracting...99.0 % complete.


'\nNUM_WORKERS = 4\nprocesses = [multiprocessing.Process(target=a.extractMelody(pathToTrack,tList)) for _ in range(NUM_WORKERS)]\nprint "Starting Process"\n[process.start() for process in processes]\nprint "Joining process"\n[process.join() for process in processes]\n'